In [1]:
import cv2, os, glob, re, json
import numpy as np
import math
import tensorflow as tf



def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

def warp_image(img, warp_shape, src, dst):

    # Get the perspective transformation matrix and its inverse
    M = cv2.getPerspectiveTransform(src, dst)
    invM = cv2.getPerspectiveTransform(dst, src)

    # Warp the image
    warped = cv2.warpPerspective(img, M, warp_shape, flags=cv2.INTER_AREA)
    return warped, M, invM
def crop_image(img, crop_box):
    cropped_img = img[crop_box[1]:crop_box[3], crop_box[0]:crop_box[2]]
    return cropped_img
#Calculate the directional gradient
def abs_sobel_thresh(img_channel, orient='x', sobel_kernel=3 , thresh_min=0, thresh_max=255):
    if orient == 'x':
        sobel = cv2.Sobel(img_channel, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    else:
        sobel = cv2.Sobel(img_channel, cv2.CV_64F, 0, 1, ksize=sobel_kernel)

    abs_sobel = np.absolute(sobel)
    binary_output = np.uint8(255*abs_sobel/np.max(abs_sobel))
    threshold_mask = np.zeros_like(binary_output)
    threshold_mask[(binary_output >= thresh_min) & (binary_output <= thresh_max)] = 1
    return threshold_mask

#Calculate the gradient magitude
def mag_thresh(img_channel, sobel_kernel=3, thresh_min=0, thresh_max=255):

    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(img_channel, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(img_channel, cv2.CV_64F, 0, 1, ksize=sobel_kernel)

    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)

    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255
    gradmag = (gradmag/scale_factor).astype(np.uint8)

    # Create a binary mask of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= thresh_min) & (gradmag <= thresh_max)] = 1

    return binary_output

def value_thresh(image_channel, thresh_min=0, thresh_max=255):
    binary_mask = np.zeros_like(image_channel)
    binary_mask[(image_channel > thresh_min) & (image_channel <= thresh_max)] = 1

    return binary_mask

def colour_thresh(image, thresh_min= np.array([0,0,0]), thresh_max = np.array([255,255,255])):
    colour_mask = cv2.inRange(image, thresh_min, thresh_max)/255
    return colour_mask.astype(np.uint8)


#Calculate the gradient direction
def dir_threshold(img_channel, sobel_kernel=3, thresh_min=0, thresh_max=np.pi/2):

    # Take both Sobel x and y gradients
    sobel_x = cv2.Sobel(img_channel, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobel_y = cv2.Sobel(img_channel, cv2.CV_64F, 0, 1, ksize=sobel_kernel)

    # Take the absolute value of the x and y gradients
    abs_sobel_x = np.absolute(sobel_x)
    abs_sobel_y = np.absolute(sobel_y)

    # Calculate the direction of the gradient
    direction = np.arctan2(abs_sobel_y, abs_sobel_x)
    direction = np.absolute(direction)

    # Binary mask where the griadent directions are within the threshold values
    mask = np.zeros_like(direction)
    mask[(direction >= thresh_min) & (direction <= thresh_max)] = 1

    return mask

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)
    

    #filling pixels inside the polygon defined by "vertices" with 1
    cv2.fillPoly(mask, vertices, 1)

    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def detect_edges(image, cut_point,start_point, debug_channels=False):
    """Function to output a binary threshold image for detecting white lanes"""
    ysize = image.shape[0]
    xsize = image.shape[1]
    crop_box = (20, cut_point, xsize - 20, ysize - start_point)
    image = crop_image(image, crop_box)

    # Get image dimensions.
    height = image.shape[0]
    width = image.shape[1]

    vertices = np.array([[(120, 0), (width/2-42, height-40), (width/2-42, height),
                          (width/2+42, height), (width/2+42, height-40), (width-120, 0)]], dtype=np.int32)

    #############
    # Track 1, colour mask filtering
    ##############
    # Convert to HSL colour space
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

    # Adjust parameters for white filtering in HSV
    hsv_white_low = np.array([0, 0, 200])  # Lower threshold for white color
    hsv_white_high = np.array([255, 30, 255])  # Upper threshold for white color

    white_mask = colour_thresh(hsv, hsv_white_low, hsv_white_high)
    white_mask = region_of_interest(white_mask, vertices)

    # Filter based on brightness (value channel in HSV)
    value_channel = hsv[:, :, 2]
    brightness_threshold = 200  # Adjust this threshold as needed
    value_mask = (value_channel >= brightness_threshold).astype(np.uint8)

    # Combine the white mask and brightness mask
    combined_mask = cv2.bitwise_and(white_mask, value_mask)

    ########################
    # Track 2: gradient filtering in HLS space
    ##########################
    img_l = hls[:, :, 1]
    img_l_grad_x = abs_sobel_thresh(img_l, 'x', 5, 120, 255)
    img_l_grad_y = abs_sobel_thresh(img_l, 'y', 5, 120, 255)

    grad_l_mask = np.zeros_like(img_l)
    grad_l_mask[(img_l_grad_x == 1) | (img_l_grad_y == 1)] = 1

    img_s = hls[:, :, 2]
    img_s_grad_x = abs_sobel_thresh(img_s, 'x', 5, 120, 255)
    img_s_grad_y = abs_sobel_thresh(img_s, 'y', 5, 120, 255)

    grad_s_mask = np.zeros_like(img_s)
    grad_s_mask[(img_s_grad_x == 1) | (img_s_grad_y == 1)] = 1

    # Combine the gradient filters for the two channels
    gradient_mask = np.zeros_like(grad_s_mask)
    gradient_mask[(grad_s_mask == 1) | (grad_l_mask == 1)] = 1

    gradient_mask = region_of_interest(gradient_mask, vertices)

    if debug_channels:
        return np.dstack((np.zeros_like(combined_mask), combined_mask, gradient_mask)) * 255
    else:
        binary_mask = np.zeros_like(combined_mask)
        binary_mask[(combined_mask == 1) | (gradient_mask == 1)] = 1
        return binary_mask

In [2]:
def rectify_real(img,plot):
  ysize = img.shape[0]
  xsize = img.shape[1]
  print(xsize)
  undist = img
  if(plot):
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
  side_top=33
  side_bottom=25
  height_bottom=27
  src = np.float32([
      ((xsize/2)-side_top,0),
      ((xsize/2)+side_top,0),
      (xsize-side_bottom,ysize-height_bottom),
      (side_bottom,ysize-height_bottom)
  ])
  src2=src.astype(np.int32)

  new_side=134
  new_height=0
  dst = np.float32([
      (new_side, 0),
      (xsize-new_side, 0),
      (xsize-new_side, ysize-new_height),
      (new_side, ysize-new_height)
      ])
  dst2=dst.astype(np.int32)

  isClosed = True
  color = (255, 0, 0)
  color2 = (255, 0, 0)
  thickness = 2
  undist2=undist.copy()
  undist2 = cv2.polylines(undist2, [src2],
                      isClosed, color, thickness)
  if(plot):
    axes[0].imshow(undist2)
  warped, M, invM = warp_image(undist, (xsize, ysize), src, dst)
  warped2=warped.copy()
  warped2 = cv2.polylines(warped2, [dst2],
                      isClosed, color2, thickness)
  if(plot):
    axes[1].imshow(warped2)

  plt.show
  return undist2,warped,warped2

def unrectify_real(warped,undist):
    ysize, xsize = undist.shape[:2]
    print(ysize,xsize)
    warped = cv2.resize(warped, (xsize, ysize), interpolation=cv2.INTER_LINEAR)
    side_top=33
    new_side=134
    new_height=0
    side_top=33
    side_bottom=25
    height_bottom=27
    src = np.float32([
      (new_side, 0),
      (xsize-new_side, 0),
      (xsize-new_side, ysize-new_height),
      (new_side, ysize-new_height)
    ])
    dst = np.float32([
      ((xsize/2)-side_top,0),
      ((xsize/2)+side_top,0),
      (xsize-side_bottom,ysize),
      (side_bottom,ysize)
    ])
    unwarped, M, invM = warp_image(warped, (xsize, ysize), src, dst)
    

    return unwarped

def unrectify_sim(warped,undist):
    ysize, xsize = undist.shape[:2]
    print(ysize,xsize)
    # warped = cv2.resize(warped, (xsize, ysize), interpolation=cv2.INTER_LINEAR)
    side_top=13
    side_bottom=5
    height_bottom=13
    src = np.float32([
      ((xsize/2)-side_top,0),
      ((xsize/2)+side_top,0),
      (xsize-side_bottom,ysize),
      (side_bottom,ysize)
    ])
    src2=src.astype(np.int32)
    
    new_side=((xsize)/2)-side_top*2
    print(new_side)
    new_height=0
    dst = np.float32([
      (new_side, 0),
      (xsize-new_side, 0),
      (xsize-new_side, ysize-new_height),
      (new_side, ysize-new_height)
      ])
    dst2=dst.astype(np.int32)
    unwarped, M, invM = warp_image(warped, (xsize, ysize), dst, src)
    

    return unwarped

def rectify_sim(img,plot):
  img
  ysize = img.shape[0]
  xsize = img.shape[1]
  print(xsize)
  undist = img
  if(plot):
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
  side_top=13
  side_bottom=5
  height_bottom=13
  src = np.float32([
      ((xsize/2)-side_top,0),
      ((xsize/2)+side_top,0),
      (xsize-side_bottom,ysize-height_bottom),
      (side_bottom,ysize-height_bottom)
  ])
  src2=src.astype(np.int32)

  new_side=((xsize)/2)-side_top*2
  print(new_side)
  new_height=0
  dst = np.float32([
      (new_side, 0),
      (xsize-new_side, 0),
      (xsize-new_side, ysize-new_height),
      (new_side, ysize-new_height)
      ])
  dst2=dst.astype(np.int32)

  isClosed = True
  color = (255, 0, 0)
  color2 = (255, 0, 0)
  thickness = 2
  undist2=undist.copy()
  undist2 = cv2.polylines(undist2, [src2],
                      isClosed, color, thickness)
  if(plot):
    axes[0].imshow(undist2)
  warped, M, invM = warp_image(undist, (xsize, ysize), src, dst)
  warped2=warped.copy()
  warped2 = cv2.polylines(warped2, [dst2],
                      isClosed, color2, thickness)
  if(plot):
    axes[1].imshow(warped2)

  plt.show
  return undist2,warped,warped2
def find_lane_masks(img,img_type,cut_point,start_point):
  ysize = img.shape[0]
  xsize = img.shape[1]
  crop_box = (20, cut_point, xsize-20, ysize-start_point)
  crop_box = (20, cut_point, xsize-20, ysize-start_point)
  img=crop_image(img, crop_box)
  # fig, axes = plt.subplots(1, 3, figsize=(3, 5))
  img_hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
  img_hls = cv2.medianBlur(img_hls, 5)

  big_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (31, 31))
  small_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))

  not_black = (img_hls[:, :, 2].astype(np.uint8) > 1)
  if img_type==0:
   white = cv2.inRange(img, (100, 150, 100), (255, 255, 255))
  elif img_type==1:
   white = cv2.inRange(img, (100, 120, 140), (220, 220, 220))
  else:
   white = cv2.inRange(img, (100, 120, 120), (220, 220, 220))


  if img_type==0:
   yellow = cv2.inRange(img, (100, 100, 0), (255, 255, 100))
  elif img_type==1:
   yellow = cv2.inRange(img, (100, 100, 0), (255, 255, 100))
  else:
   yellow = cv2.inRange(img, (100, 120, 0), (255, 255, 120))

  # axes[0].imshow(img)
  # axes[1].imshow(white)
  # axes[2].imshow(yellow)
  # fig.show()
  return white,yellow
def get_poly_points(img,left_fit, right_fit):
    '''
    Get the points for the left lane/ right lane defined by the polynomial coeff's 'left_fit'
    and 'right_fit'
    :param left_fit (ndarray): Coefficients for the polynomial that defines the left lane line
    :param right_fit (ndarray): Coefficients for the polynomial that defines the right lane line
    : return (Tuple(ndarray, ndarray, ndarray, ndarray)): x-y coordinates for the left and right lane lines
    '''
    ysize = img.shape[0]
    xsize = img.shape[1]


    # Get the points for the entire height of the image
    plot_y = np.linspace(0, ysize-1, ysize)
    plot_xleft = left_fit[0] * plot_y**2 + left_fit[1] * plot_y + left_fit[2]
    plot_xright = right_fit[0] * plot_y**2 + right_fit[1] * plot_y + right_fit[2]

    # But keep only those points that lie within the image
    plot_xleft = plot_xleft[(plot_xleft >= 0) & (plot_xleft <= xsize - 1)]
    plot_xright = plot_xright[(plot_xright >= 0) & (plot_xright <= xsize - 1)]
    plot_yleft = np.linspace(ysize - len(plot_xleft), ysize - 1, len(plot_xleft))
    plot_yright = np.linspace(ysize - len(plot_xright), ysize - 1, len(plot_xright))

    return plot_xleft.astype(np.int), plot_yleft.astype(np.int), plot_xright.astype(np.int), plot_yright.astype(np.int)

def check_validity(img,left_fit, right_fit, diagnostics=False):
    '''
    Determine the validity of lane lines represented by a set of second order polynomial coefficients
    :param left_fit (ndarray): Coefficients for the 2nd order polynomial that defines the left lane line
    :param right_fit (ndarray): Coefficients for the 2nd order polynomial that defines the right lane line
    :param diagnostics (boolean): Boolean flag for logging
    : return (boolean)
    '''

    ysize = img.shape[0]
    xsize = img.shape[1]
    
    if left_fit is None or right_fit is None:
        print("I am out")
        return False
    else:
        print(left_fit)
        print(right_fit)
    print("I am in")
    
    plot_xleft, plot_yleft, plot_xright, plot_yright = get_poly_points(img,left_fit, right_fit)

    # Check whether the two lines lie within a plausible distance from one another for three distinct y-values

    y1 = ysize - 1 # Bottom
    y2 = ysize - int(min(len(plot_yleft), len(plot_yright)) * 0.35) # For the 2nd and 3rd, take values between y1 and the top-most available value.
    y3 = ysize - int(min(len(plot_yleft), len(plot_yright)) * 0.75)

    # Compute the respective x-values for both lines
    x1l = left_fit[0]  * (y1**2) + left_fit[1]  * y1 + left_fit[2]
    x2l = left_fit[0]  * (y2**2) + left_fit[1]  * y2 + left_fit[2]
    x3l = left_fit[0]  * (y3**2) + left_fit[1]  * y3 + left_fit[2]

    x1r = right_fit[0] * (y1**2) + right_fit[1] * y1 + right_fit[2]
    x2r = right_fit[0] * (y2**2) + right_fit[1] * y2 + right_fit[2]
    x3r = right_fit[0] * (y3**2) + right_fit[1] * y3 + right_fit[2]

    # Compute the L1 norms
    x1_diff = abs(x1l - x1r)
    x2_diff = abs(x2l - x2r)
    x3_diff = abs(x3l - x3r)

    # Define the threshold values for each of the three points
    min_dist_y1 = 1 # 510 # 530
    max_dist_y1 = 730 # 750 # 660
    min_dist_y2 = 1
    max_dist_y2 = 730 # 660
    min_dist_y3 = 1
    max_dist_y3 = 730 # 660

    if (x1_diff < min_dist_y1) | (x1_diff > max_dist_y1) | \
        (x2_diff < min_dist_y2) | (x2_diff > max_dist_y2) | \
        (x3_diff < min_dist_y3) | (x3_diff > max_dist_y3):
        if diagnostics:
            print("Violated distance criterion: " +
                  "x1_diff == {:.2f}, x2_diff == {:.2f}, x3_diff == {:.2f}".format(x1_diff, x2_diff, x3_diff))
        return False

    # Check whether the line slopes are similar for two distinct y-values
    # x = Ay**2 + By + C
    # dx/dy = 2Ay + B

    y1left_dx  = 2 * left_fit[0]  * y1 + left_fit[1]
    y3left_dx  = 2 * left_fit[0]  * y3 + left_fit[1]
    y1right_dx = 2 * right_fit[0] * y1 + right_fit[1]
    y3right_dx = 2 * right_fit[0] * y3 + right_fit[1]

    # Compute the L1-norm
    norm1 = abs(y1left_dx - y1right_dx)
    norm2 = abs(y3left_dx - y3right_dx)

#     if diagnostics: print( norm1, norm2)

    # Define the L1 norm threshold
    thresh = 5 #0.58
    if (norm1 >= thresh) | (norm2 >= thresh):
        if diagnostics:
            print("Violated tangent criterion: " +
                  "norm1 == {:.3f}, norm2 == {:.3f} (thresh == {}).".format(norm1, norm2, thresh))
            return False

    return True

def polyfit_sliding_window(binary,img,cut_point,start_point, lane_width_px=1, visualise=False, diagnostics=True):
    '''
    Detect lane lines in a thresholded binary image using the sliding window technique
    :param binary (ndarray): Thresholded binary image
    :param lane_width_px (int): Average lane line width (in px) for the warped image
    computed empirically
    :param visualise (boolean): Boolean flag for visualisation
    :param diagnositics (boolean): Boolean flag for logging
    '''

    # global cache
    ret = True

    ysize = binary.shape[0]-start_point
    xsize = binary.shape[1]

    # Sanity check
    if binary.max() <= 0:
        return False,img, out, np.array([]), np.array([]),0

    # Step 1: Compute the histogram along all the columns in the lower half of the image.
    # The two most prominent peaks in this histogram will be good indicators of the
    # x-position of the base of the lane lines
    histogram = None
    cutoffs = [int(binary.shape[0] / 2), 0]

    for cutoff in cutoffs:
        histogram = np.sum(binary[cutoff:, :], axis=0)

        if histogram.max() > 0:
            break

    if histogram.max() == 0:
        print('Unable to detect lane lines in this frame. Trying another frame!')
        return False,img, out, np.array([]), np.array([]),0

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0] / 2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    center_base=leftx_base+(rightx_base-leftx_base)/2

    if visualise:
        plot_images([(binary, 'Binary')])
        plt.plot(histogram, 'm', linewidth=4.0)
        plt.plot(histogram, 'm', linewidth=4.0)
        plt.plot((center_base, center_base), (0, ysize), 'c')
        plt.plot((0, xsize), (cutoff, cutoff), 'c')

    out = np.dstack((binary, binary, binary)) * 255
    print(out.shape[0])

    # TODO: HERE

    nb_windows = 20 # number of sliding windows
    margin = 20 # width of the windows +/- margin
    minpix = 5 # min number of pixels needed to recenter the window
    window_height = int(ysize / nb_windows)
    min_lane_pts = 50  # min number of 'hot' pixels needed to fit a 2nd order polynomial as a
                    # lane line

    # Identify the x-y positions of all nonzero pixels in the image
    # Note: the indices here are equivalent to the coordinate locations of the
    # pixel
    nonzero = binary.nonzero()
    nonzerox = np.array(nonzero[1])
    nonzeroy = np.array(nonzero[0])

    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    for window in range(nb_windows+10):
        # Identify window boundaries in x and y (and left and right)
        win_y_low = ysize - (1 + window) * window_height
        win_y_high = ysize - window * window_height

        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin

        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        # Draw windows for visualisation
        cv2.rectangle(out, (win_xleft_low, win_y_low), (win_xleft_high, win_y_high),\
                      (0, 255, 0), 2)
        cv2.rectangle(out, (win_xright_low, win_y_low), (win_xright_high, win_y_high),\
                      (0, 255, 0), 2)
        # cv2.rectangle(img, (win_xleft_low, win_y_low), (win_xleft_high, win_y_high),\
        #               (0, 255, 0), 2)
        # cv2.rectangle(img, (win_xright_low, win_y_low), (win_xright_high, win_y_high),\
        #               (0, 255, 0), 2)

        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy <= win_y_high)
                         & (nonzerox >= win_xleft_low) & (nonzerox <= win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy <= win_y_high)
                         & (nonzerox >= win_xright_low) & (nonzerox <= win_xright_high)).nonzero()[0]

        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) >  minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))

        if len(good_right_inds) > minpix:
            rightx_current = int(np.mean(nonzerox[good_right_inds]))

    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract pixel positions for the left and right lane lines
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    left_fit, right_fit = None, None

    # Sanity check; Fit a 2nd order polynomial for each lane line pixels
    if len(leftx) >= min_lane_pts and len(rightx) >= min_lane_pts:
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)

    # Validate detected lane lines
    valid = check_validity(binary,left_fit, right_fit, diagnostics=diagnostics)

    if not valid:
        # If the detected lane lines are NOT valid:
        # 1. Compute the lane lines as an average of the previously detected lines
        # from the cache and flag this detection cycle as a failure by setting ret=False
        # 2. Else, if cache is empty, return


        ret = False
    else:
        plot_xleft, plot_yleft, plot_xright, plot_yright = get_poly_points(binary,left_fit, right_fit)
    
        # Color the detected pixels for each lane line
        out[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [255, 0, 0]
    
        left_poly_pts = np.array([np.transpose(np.vstack([plot_xleft, plot_yleft]))])
        right_poly_pts = np.array([np.transpose(np.vstack([plot_xright, plot_yright]))])
    
        cv2.polylines(out, np.int32([left_poly_pts]), isClosed=False, color=(255,255,0), thickness=4)
        cv2.polylines(out, np.int32([right_poly_pts]), isClosed=False, color=(255,255,0), thickness=4)
    
        left_poly_pts = np.array([np.transpose(np.vstack([plot_xleft, plot_yleft+cut_point-start_point]))])
        right_poly_pts = np.array([np.transpose(np.vstack([plot_xright, plot_yright+cut_point-start_point]))])
    
        # # Plot the fitted polynomial
        
        cv2.polylines(img, np.int32([left_poly_pts]), isClosed=False, color=(255,255,0), thickness=4)
        cv2.polylines(img, np.int32([right_poly_pts]), isClosed=False, color=(255,255,0), thickness=4)

    if visualise:
        plot_images([(img, 'Original'), (out, 'Out')], figsize=(30, 40))

    return ret, img, out, np.array([left_fit, right_fit]),histogram,center_base

def plot_images(data, layout='row', cols=2, figsize=(20, 12)):
    '''
    Utility function for plotting images
    :param data [(ndarray, string)]: List of data to display, [(image, title)]
    :param layout (string): Layout, row-wise or column-wise
    :param cols (number): Number of columns per row
    :param figsize (number, number): Tuple indicating figure size
    '''
    rows = math.ceil(len(data) / cols)
    f, ax = plt.subplots(figsize=figsize)
    if layout == 'row':
        for idx, d in enumerate(data):
            img, title = d

            plt.subplot(rows, cols, idx+1)
            plt.title(title, fontsize=20)
            plt.axis('off')
            if len(img.shape) == 2:
                plt.imshow(img, cmap='gray')

            elif len(img.shape) == 3:
                plt.imshow(img)

    elif layout == 'col':
        counter = 0
        for r in range(rows):
            for c in range(cols):
                img, title = data[r + rows*c]
                nb_channels = len(img.shape)

                plt.subplot(rows, cols, counter+1)
                plt.title(title, fontsize=20)
                plt.axis('off')
                if len(img.shape) == 2:
                    plt.imshow(img, cmap='gray')

                elif len(img.shape) == 3:
                    plt.imshow(img)

                counter += 1

    return ax

In [3]:
def keep_only_lines(image):
    # Convert the image to grayscale
    # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray=image
    # Apply Gaussian blur to reduce noise and improve line detection
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Detect lines using Hough Line Transform
    lines = cv2.HoughLinesP(blurred, 1, np.pi / 180, threshold=10, minLineLength=2, maxLineGap=1000)

    if lines is not None:
        # Create a blank canvas with the same dimensions as the original image
        line_image = np.zeros_like(image)

        # Draw the lines on the blank canvas
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green lines

        return line_image
    else:
        return None

In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import matplotlib.image as mpimg
from PIL import Image, ImageOps

def plot_images_side_by_side(image_paths_list, titles_list,cut_point,start_point):
    num_images = len(image_paths_list)

    fig, axes = plt.subplots(3, num_images, figsize=(3, 3))
    plt.tight_layout()

    for i, (image_path, title) in enumerate(zip(image_paths_list, titles_list)):
        image = mpimg.imread(image_path)
        

        if (image_path.split(".")[-1]=="png"):
          image=image[:,:,:3]
          image = (image * 255).astype(np.uint8)
        
        


        axes[0,i].imshow(image)
        axes[0,i].set_title(title+" "+image_path.split("/")[-1])
        axes[0,i].axis('off')
        
        
        if(i!=0):
            road = cv2.inRange(image, (0,0, 0), (150, 100, 100))
        else:
            road = cv2.inRange(image, (20, 20, 20), (80, 80, 80))

        if(i!=0):
            road2 = cv2.inRange(image, (100, 60, 60), (150, 110, 100))
        else:
            road2 = cv2.inRange(image, (20, 20, 20), (80, 80, 80))

        if i==0:
           yellow = cv2.inRange(image, (100, 100, 0), (255, 255, 100))
        else:
           yellow = cv2.inRange(image, (80, 80, 0), (255, 255, 80))

        if i==0:
           yellow2 = cv2.inRange(image, (100, 100, 0), (255, 255, 100))
        else:
           yellow2 = cv2.inRange(image, (150, 110, 0), (190, 140, 110))


        if i==0:
           white = cv2.inRange(image, (100, 150, 100), (255, 255, 255))
        else:
           white = cv2.inRange(image, (100, 120, 120), (220, 220, 220))

        if i==0:
           white_no = cv2.inRange(image, (170, 170, 170), (255, 255, 255))
        else:
           white_no = cv2.inRange(image, (170, 170, 170), (255, 255, 255))

        if i==0:
           white_no2 = cv2.inRange(image, (81, 81, 81), (82, 82, 82))
        else:
           white_no2 = cv2.inRange(image, (80, 60, 60), (255, 255, 255))

        height, width = road.shape
        vertices = np.array([[(0, height),(width/2-70,0),(width/2,0),(width/2+110,0),(width,height)]], dtype=np.int32)
        gray_region = np.ones((height, width, 1), dtype=np.uint8)
        gray_region=region_of_interest(gray_region, vertices)

        if i==0:
           gray = cv2.inRange(image, (50, 50, 50), (82, 82, 82))
           gray_selective=gray
        else:
           gray = cv2.inRange(image, (100, 60, 70), (130, 100, 110))
           gray_selective=gray
           gray_selective[gray_region!=0]=0
        
        
        road_mapped=road*128
        road_mapped[road2 > 0]=128
        road_mapped[white_no > 0]=0
        road_mapped[white > 0]=0
        #road_mapped[gray_selective > 0]=0
        
        height, width = road.shape
        merged_mask = image
        merged_mask[road_mapped == 128]=[255,0,0]
        merged_mask[road_mapped == 64]=[0,255,0]
        merged_mask[yellow > 0]=[0,0,255]
        merged_mask[yellow2 > 0]=[0,0,255]
        

        
        mask_output = np.zeros((height, width, 1), dtype=np.uint8)
        mask_output[road > 0]=128
        mask_output[road2 > 0]=128
        mask_output[white_no > 0]=0
        mask_output[white > 0]=0
        #mask_output[gray_selective > 0]=0
        mask_output[yellow > 0]=255
        mask_output[yellow2 > 0]=255

        axes[1,i].imshow(mask_output)
        axes[1,i].set_title(title+" mask "+image_path.split("/")[-1])
        axes[1,i].axis('off')

        axes[2,i].imshow(merged_mask)
        axes[2,i].set_title(title+" overlay mask "+image_path.split("/")[-1])
        axes[2,i].axis('off')
        
        
        

        
       
        # if i==0:
        #      png_image = tf.image.encode_png(mask_output)
        #      os.makedirs('./content/datasets/processed_data_testing/sim_mask_cv/', exist_ok=True)
        #      with open('./content/datasets/processed_data_testing/sim_mask_cv/'+image_path.split("0001_")[-1], 'wb') as f:
        #        f.write(png_image.numpy())
        if i==1:
            png_image = tf.image.encode_png(mask_output)
            os.makedirs('./content/datasets/processed_data_testing/real_mask_cv/', exist_ok=True)
            with open('./content/datasets/processed_data_testing/real_mask_cv/'+image_path.split("0001_")[-1], 'wb') as f:
              f.write(png_image.numpy())
        
   
    return image_path.split("image_")[-1]
    

# Provide paths to the image folders and collect image file paths
folder1_path = './content/datasets/processed_data_testing/sim/'
folder2_path = './content/datasets/processed_data_testing/real/'


# folder1_path = './content/datasets/data_in/objects/sim/'
# folder2_path = './content/datasets/data_in/objects/real/'
# folder3_path = './content/datasets/data_in/objects/real/'

folder1_images = sorted([os.path.join(folder1_path, filename) for filename in os.listdir(folder1_path) if filename.endswith('.png')],key=natural_sort_key)
folder2_images = sorted([os.path.join(folder2_path, filename) for filename in os.listdir(folder2_path) if filename.endswith('.png')],key=natural_sort_key)

# Assuming each folder has the same number of images and the same filenames
num_images = len(folder1_images)
print(num_images)
titles = ["sim","real_strict"]

cut_point=100
start_point=1

# Combine image lists from all folders
all_images = list(zip(folder2_images, folder2_images))

# Plot the images side by side
count=0
id_list=[]
for images_tuple in all_images:
    id=plot_images_side_by_side(images_tuple, titles,cut_point,start_point)
    id_list.append(id)